# <div align="center" style="color: #ff5733;">New KPI Report</div>

## *Requestor is Jennyfer & Anne*

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
# path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
# Example: pd.set_option('display.max_columns', None)

c:\Users\DwaipayanChakroborti\Myenv\reporting\lib\site-packages\google\auth\_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


# FPD30 December 2024 Month End

In [5]:
sq = """SELECT t1.new_loan_type,
case when sum(t1.obsFPD30) = 0 then 0 else ROUND(SUM(CASE when t1.defFPD30 = 1 then 1 ELSE 0 end )/sum(t1.obsFPD30)*100,3) end as percentage_of_FPD30
from `prj-prod-dataplatform.risk_credit_mis.loan_master_table` t1
inner JOIN prj-prod-dataplatform.risk_credit_mis.loan_bucket_flow_report_core t2
ON
t1.loanAccountNumber = t2.loanAccountNumber
WHERE t1.flagDisbursement = 1
and t1.new_loan_type in ('Quick','Flex','SIL-Instore', 'SIL ZERO', 'SIL Competitor', 'SIL Repeat')
and t2.loanStatus in ('In Arrears','Normal')
and t2.bucketDate in ('2024-12-31')
GROUP BY 1;"""

dec24fpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')
dec24fpd30

Job ID 7f66a432-e54d-4fe9-9206-237b54e3ada0 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,new_loan_type,percentage_of_FPD30
0,Quick,14.192
1,SIL ZERO,7.251
2,SIL Competitor,5.983
3,Flex,0.313
4,SIL-Instore,7.858
5,SIL Repeat,0.000


# 30+, 60+, 90+ for December 2024

In [6]:
sq = """with b as
(select t1.new_loan_type 
, count(distinct t2.loanAccountNumber) cnt_loan
, count(distinct case when t2.Max_current_DPD >=30 then t2.loanAccountNumber end) thirtyplus
, count(distinct case when date_diff(date(t2.bucketDate), date(t2.firstduedate), day) >=30 then t2.loanAccountNumber end) obsthirtyplus
, count(distinct case when t2.Max_current_DPD >=60 then t2.loanAccountNumber end) sixtyplus
, count(distinct case when date_diff(date(t2.bucketDate), date(t2.firstduedate), day) >=60 then t2.loanAccountNumber end) obssixtyplus
, count(distinct case when t2.Max_current_DPD >=90 then t2.loanAccountNumber end) nintyplus
, count(distinct case when date_diff(date(t2.bucketDate), date(t2.firstduedate), day) >= 90 then t2.loanAccountNumber end) obsnintyplus
from `risk_credit_mis.loan_bucket_flow_report_core` t2
inner join `risk_credit_mis.loan_master_table` t1 on t2.loanAccountNumber = t1.loanAccountNumber
WHERE t1.flagDisbursement = 1
and t1.new_loan_type in ('Quick','Flex','SIL-Instore', 'SIL ZERO', 'SIL Competitor', 'SIL Repeat')
and t2.loanStatus in ('In Arrears','Normal')
and t2.bucketDate in ('2024-12-31')
group by 1
)
select 
new_loan_type,
case when sum(obsthirtyplus) = 0 then 0 else round(sum(thirtyplus)/sum(obsthirtyplus)*100, 3) end thirtyplus,
case when sum(obsthirtyplus) = 0 then 0 else round(sum(sixtyplus)/sum(obsthirtyplus)*100, 3) end sixtyplus,
case when sum(obsnintyplus) = 0 then 0 else round(sum(nintyplus)/sum(obsnintyplus)*100, 3) end nintyplus,
from b
group by 1
order by 1
;"""

secondpartDec24 = client.query(sq).to_dataframe(progress_bar_type='tqdm')
secondpartDec24

Job ID e7c87396-9f9e-49da-ac80-4ccb6d3f3561 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,new_loan_type,thirtyplus,sixtyplus,nintyplus
0,Flex,34.742,29.421,24.570
1,Quick,31.646,22.617,29.956
2,SIL Competitor,0.000,0.000,0.000
3,SIL Repeat,0.000,0.000,0.000
4,SIL ZERO,12.354,6.254,11.726
5,SIL-Instore,26.568,20.447,23.323


# 30+, 60+, 90+ for December 2024 (without Product)

In [7]:
sq = """with b as
(select 
count(distinct t2.loanAccountNumber) cnt_loan
, count(distinct case when t2.Max_current_DPD >=30 then t2.loanAccountNumber end) thirtyplus
, count(distinct case when date_diff(date(t2.bucketDate), date(t2.firstduedate), day) >=30 then t2.loanAccountNumber end) obsthirtyplus
, count(distinct case when t2.Max_current_DPD >=60 then t2.loanAccountNumber end) sixtyplus
, count(distinct case when date_diff(date(t2.bucketDate), date(t2.firstduedate), day) >=60 then t2.loanAccountNumber end) obssixtyplus
, count(distinct case when t2.Max_current_DPD >=90 then t2.loanAccountNumber end) nintyplus
, count(distinct case when date_diff(date(t2.bucketDate), date(t2.firstduedate), day) >= 90 then t2.loanAccountNumber end) obsnintyplus
from `risk_credit_mis.loan_bucket_flow_report_core` t2
inner join `risk_credit_mis.loan_master_table` t1 on t2.loanAccountNumber = t1.loanAccountNumber
WHERE t1.flagDisbursement = 1
and t1.new_loan_type in ('Quick','Flex','SIL-Instore', 'SIL ZERO', 'SIL Competitor', 'SIL Repeat')
and t2.loanStatus in ('In Arrears','Normal')
and t2.bucketDate in ('2024-12-31')
)
select 
case when sum(obsthirtyplus) = 0 then 0 else round(sum(thirtyplus)/sum(obsthirtyplus)*100, 3) end as thirtyplus,
case when sum(obsthirtyplus) = 0 then 0 else round(sum(sixtyplus)/sum(obsthirtyplus)*100, 3) end as sixtyplus,
case when sum(obsnintyplus) = 0 then 0 else round(sum(nintyplus)/sum(obsnintyplus)*100, 3) end as nintyplus,
from b
order by 1
;"""

thirdpartDec24 = client.query(sq).to_dataframe(progress_bar_type='tqdm')
thirdpartDec24

Job ID 53b8928e-5960-4934-961f-4a20097b597d successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,thirtyplus,sixtyplus,nintyplus
0,26.864,20.309,24.112


In [8]:
dec24fpd30.head()

,new_loan_type,percentage_of_FPD30
0,Quick,14.192
1,SIL ZERO,7.251
2,SIL Competitor,5.983
3,Flex,0.313
4,SIL-Instore,7.858


In [9]:
secondpartDec24.head()

,new_loan_type,thirtyplus,sixtyplus,nintyplus
0,Flex,34.742,29.421,24.570
1,Quick,31.646,22.617,29.956
2,SIL Competitor,0.000,0.000,0.000
3,SIL Repeat,0.000,0.000,0.000
4,SIL ZERO,12.354,6.254,11.726


In [10]:
dfDec24 = dec24fpd30.merge(secondpartDec24, on='new_loan_type', how='left')
dfDec24

,new_loan_type,percentage_of_FPD30,thirtyplus,sixtyplus,nintyplus
0,Quick,14.192,31.646,22.617,29.956
1,SIL ZERO,7.251,12.354,6.254,11.726
2,SIL Competitor,5.983,0.000,0.000,0.000
3,Flex,0.313,34.742,29.421,24.570
4,SIL-Instore,7.858,26.568,20.447,23.323
5,SIL Repeat,0.000,0.000,0.000,0.000


In [11]:
thirdpartDec24.head()

,thirtyplus,sixtyplus,nintyplus
0,26.864,20.309,24.112


In [12]:
resultDec24 = pd.concat([dfDec24, thirdpartDec24], ignore_index=True)


In [13]:
resultDec24['new_loan_type'] = resultDec24['new_loan_type'].fillna("Portfolio")
resultDec24

,new_loan_type,percentage_of_FPD30,thirtyplus,sixtyplus,nintyplus
0,Quick,14.192,31.646,22.617,29.956
1,SIL ZERO,7.251,12.354,6.254,11.726
2,SIL Competitor,5.983,0.000,0.000,0.000
3,Flex,0.313,34.742,29.421,24.570
4,SIL-Instore,7.858,26.568,20.447,23.323
5,SIL Repeat,0.000,0.000,0.000,0.000
6,Portfolio,NaN,26.864,20.309,24.112


In [14]:
import pandas as pd
from openpyxl import load_workbook


In [16]:
# Load the existing Excel file
existing_excel_file = r'C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Collection\Anne\Jennifer Zaldivar\KRI_New_Report\KRI_Report_New\Data\20241202_KRI_Report_New.xlsx'
xls = pd.ExcelFile(existing_excel_file)

# Read each existing sheet into separate dataframes
df_january24 = pd.read_excel(xls, sheet_name='January2024')
df_february24 = pd.read_excel(xls, sheet_name='February2024')
df_march24 = pd.read_excel(xls, sheet_name='March2024')
df_april24 = pd.read_excel(xls, sheet_name='April2024')
df_may24 = pd.read_excel(xls, sheet_name='May2024')
df_june24 = pd.read_excel(xls, sheet_name='June2024')
df_july24 = pd.read_excel(xls, sheet_name='July2024')
df_aug24 = pd.read_excel(xls, sheet_name='August2024')
df_sep24 = pd.read_excel(xls, sheet_name='September2024')
df_oct24 = pd.read_excel(xls, sheet_name='Octboer2024')
df_nov24 = pd.read_excel(xls, sheet_name='November2024')



In [17]:
df_nov24

,new_loan_type,percentage_of_FPD30,thirtyplus,sixtyplus,nintyplus
0,A. Flex,0.222,38.624,34.184,29.856
1,B. Quick,13.853,34.880,25.909,36.016
2,C. SIL-Instore,8.092,27.426,21.011,24.660
3,D. SIL ZERO,8.960,12.955,3.224,0.000
4,E. SIL Repeat,0.000,0.000,0.000,0.000
5,F. SIL Competitor,0.000,0.000,0.000,0.000
6,G. Portfolio,NaN,28.259,21.448,26.057


In [18]:
# Mapping function to modify 'new_loan_type' values
def modify_loan_type(loan_type):
    mapping = {'Flex': 'A. Flex', 'SIL-Instore': 'C. SIL-Instore', 'Quick': 'B. Quick', 'SIL ZERO': 'D. SIL ZERO', 'SIL Repeat':'E. SIL Repeat', 'SIL Competitor':'F. SIL Competitor', 'Portfolio': 'G. Portfolio'}
    return mapping.get(loan_type, loan_type)  # Return original value if not found

# Apply the mapping function to the 'new_loan_type' column
resultDec24['new_loan_type'] = resultDec24['new_loan_type'].apply(modify_loan_type)
resultDec24.sort_values(by='new_loan_type')

,new_loan_type,percentage_of_FPD30,thirtyplus,sixtyplus,nintyplus
3,A. Flex,0.313,34.742,29.421,24.570
0,B. Quick,14.192,31.646,22.617,29.956
4,C. SIL-Instore,7.858,26.568,20.447,23.323
1,D. SIL ZERO,7.251,12.354,6.254,11.726
5,E. SIL Repeat,0.000,0.000,0.000,0.000
2,F. SIL Competitor,5.983,0.000,0.000,0.000
6,G. Portfolio,NaN,26.864,20.309,24.112


In [19]:
# Create a Pandas Excel writer using XlsxWriter as the engine
writer = pd.ExcelWriter(r'C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Collection\Anne\Jennifer Zaldivar\KRI_New_Report\KRI_Report_New\Data\20250104_KRI_Report_New.xlsx', engine='xlsxwriter')

# Sort each DataFrame by the first column and write to a separate worksheet, ignoring the index
df_january24.sort_values(by=df_january24.columns[0]).to_excel(writer, sheet_name='January2024', index=False)
df_february24.sort_values(by=df_february24.columns[0]).to_excel(writer, sheet_name='February2024', index=False)
df_march24.sort_values(by=df_march24.columns[0]).to_excel(writer, sheet_name='March2024', index=False)
df_april24.sort_values(by=df_april24.columns[0]).to_excel(writer, sheet_name='April2024', index=False)
df_may24.sort_values(by=df_may24.columns[0]).to_excel(writer, sheet_name='May2024', index=False)
df_june24.sort_values(by=df_june24.columns[0]).to_excel(writer, sheet_name='June2024', index=False)
df_july24.sort_values(by=df_july24.columns[0]).to_excel(writer, sheet_name='July2024', index=False)
df_aug24.sort_values(by=df_aug24.columns[0]).to_excel(writer, sheet_name='August2024', index=False)
df_sep24.sort_values(by=df_sep24.columns[0]).to_excel(writer, sheet_name='September2024', index=False)
df_oct24.sort_values(by=df_oct24.columns[0]).to_excel(writer, sheet_name='October2024', index = False)
df_nov24.sort_values(by=df_nov24.columns[0]).to_excel(writer, sheet_name='November2024', index=False)
resultDec24.sort_values(by=resultDec24.columns[0]).to_excel(writer, sheet_name='December2024', index = False)


# Close the Pandas Excel writer and output the Excel file
writer.close()